### 1) Load Test Results

In [ ]:
import os
import pandas as pd
import json

log_folder = 'logs'

# Function to clean and process each line to remove noise and keep only JSON content
def preprocess_line(line):
    try:
        return json.loads(line)
    except json.JSONDecodeError:
        return None

# Get a list of all log files in the 'log' directory
log_files = [f for f in os.listdir(log_folder) if f.endswith('.log')]

# Create a dictionary to hold all dataframes
dfs = {}

for file in log_files:
    data = []
    with open(f"{log_folder}/{file}", 'r') as f:
        for line in f:
            # Preprocess each line to remove noise
            processed_line = preprocess_line(line)
            if processed_line is not None:
                data.append(processed_line)
    
    # Convert the processed data to a DataFrame
    df = pd.json_normalize(data, sep='_')

    # Remove the .log extension and use the filename as the key in the dictionary
    dfs[file[:-4]] = df

# Example of accessing a DataFrame (you can adjust based on your needs)
# for key, df in dfs.items():
    # print(f"DataFrame for {key}:")
    # print(df.head())
    

# Print the number of test runs
print(f"Number of test runs: {len(dfs)}")

# Print the name of each test run
for name in dfs.keys():
    print(name)

### 2) Test Runs

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Increase the global font size
plt.rcParams.update({'font.size': 16})

# List of fields to plot
fields = ['requests', 'failures']

# Determine the number of rows and columns for the subplots
nrows = len(fields)
ncols = len(dfs)

# Increase the height of each chart to 10 inches
fig, axs = plt.subplots(nrows, ncols, figsize=(2 * ncols * 6, nrows * 10))
fig.subplots_adjust(hspace=0.5)

# Loop over each field, each dataframe, and each axes
for i, field in enumerate(fields):
    for j, (name, df) in enumerate(dfs.items()):
        # Check if 'run_seconds' and field exist in df
        if 'run_seconds' in df.columns and field in df.columns:
            # Convert 'run_seconds' and field to numeric type and drop rows with NaN values
            df['run_seconds'] = pd.to_numeric(df['run_seconds'], errors='coerce')
            df[field] = pd.to_numeric(df[field], errors='coerce')
            df = df.dropna(subset=['run_seconds', field])

            # Create a line plot on each axes
            if ncols > 1:
                ax = axs[i, j]
            else:
                ax = axs[i]

            ax.plot(df['run_seconds'], df[field])

            # Truncate the name before the '=' character
            truncated_name = name.split('_shape')[0]

            # Set the title of the plot to the truncated name
            ax.set_title(truncated_name)

            # Set the labels for the x-axis and y-axis
            ax.set_xlabel('run_seconds')
            ax.set_ylabel(field)
        else:
            print(f"'run_seconds' or '{field}' not found in DataFrame {name}")

# Display the plots
plt.show()



### 3) Test Summary

In [ ]:
from tabulate import tabulate

# Create a list to hold the last row of each DataFrame
summary_data = []

for name, df in dfs.items():
    # Check if the DataFrame is empty
    if not df.empty:
        # Get the last row of the DataFrame
        last_row = df.iloc[-1:]
        
        # Add the name of the DataFrame as the first column
        last_row.insert(0, 'DataFrame', name)
    
        # Append the last row to the summary_data list
        summary_data.append(last_row)

# Convert the list of last rows into a DataFrame
summary_df = pd.concat(summary_data)

# Print summary_df in a tabular format
print('Execution Summary:')
print(tabulate(summary_df, headers='keys', tablefmt='psql', showindex=False))

### 4) Test Statistics

In [ ]:
import re
import json
import statistics
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

def extract_stats_from_line(line):
    match = re.search(r'Raw call stats:\s*(\[.*\])', line)
    if match:
        json_str = match.group(1)
        call_stats = json.loads(json_str)
        
        e2e_times = [entry['response_end_time'] - entry['request_start_time'] for entry in call_stats]
        
        max_e2e = round(max(e2e_times), 3)
        min_e2e = round(min(e2e_times), 3)
        avg_e2e = round(statistics.mean(e2e_times), 3)
        variance_e2e = round(statistics.variance(e2e_times), 3)
        std_dev_e2e = round(statistics.stdev(e2e_times), 3)
        percentile_95_e2e = round(statistics.quantiles(e2e_times, n=100)[94], 3)

        return {
            "max_e2e": max_e2e,
            "min_e2e": min_e2e,
            "avg_e2e": avg_e2e,
            "variance_e2e": variance_e2e,
            "std_dev_e2e": std_dev_e2e,
            "percentile_95_e2e": percentile_95_e2e,
            "e2e_times": e2e_times
        }
    else:
        return None

# Get a list of all log files in the 'log' directory
log_files = [f for f in os.listdir(log_folder) if f.endswith('.log')]

for file in log_files:
    data = []
    with open(f"{log_folder}/{file}", 'r') as f:
        for line in f:
            stats = extract_stats_from_line(line)
            if stats:
                data.append(stats)
                print(f"Statistics for {file[:-4].split('_shape')[0]}:")
                print(f"Max e2e time: {stats['max_e2e']}")
                print(f"Min e2e time: {stats['min_e2e']}")
                print(f"Average e2e time: {stats['avg_e2e']}")
                print(f"Variance of e2e time: {stats['variance_e2e']}")
                print(f"Standard deviation of e2e time: {stats['std_dev_e2e']}")
                print(f"95th percentile of e2e time: {stats['percentile_95_e2e']}")
                print()
    
    # Create DataFrame for e2e_times
    e2e_times_list = [entry['e2e_times'] for entry in data]
    for i, e2e_times in enumerate(e2e_times_list):
        df = pd.DataFrame({'e2e_times': e2e_times})
        df['index'] = df.index
        
        # Plotting
        plt.figure(figsize=(12, 6))  # Make the chart wider
        plt.plot(df['index'], df['e2e_times'])
        truncated_name = file[:-4].split('_shape')[0]
        plt.title(truncated_name, fontsize=10)  # Smaller font size for title
        plt.xlabel('Index', fontsize=8)  # Smaller font size for x-axis label
        plt.ylabel('e2e_times', fontsize=8)  # Smaller font size for y-axis label
        plt.xticks(fontsize=8)  # Smaller font size for x-axis ticks
        plt.yticks(fontsize=8)  # Smaller font size for y-axis ticks
        plt.ylim(1, 20)  # Set y-axis limits
        plt.gca().yaxis.set_major_locator(MaxNLocator(integer=True))  # Ensure y-axis has integer increments
        plt.show()